In [36]:
from google.colab import files
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Upload image
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
print("Uploaded file name:", img_path)

# Load model
model = YOLO('yolov8n.pt')

# Load image
img = cv2.imread(img_path)
if img is None:
    print("Error: Could not load image. Check the path!")
else:
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Run detection
    results = model(img_rgb)

    # Show annotated image
    annotated_img = results[0].plot()
    plt.figure(figsize=(10,6))
    plt.imshow(annotated_img)
    plt.axis('off')
    plt.title("Vehicle Detection & Classification")
    plt.show()

    height, width, _ = img_rgb.shape

    # Crop and display each detected box separately
    boxes = results[0].boxes  # detected boxes

    for i, box in enumerate(boxes):
        # Convert box coordinates (xyxy format) to integers and clamp them within image size
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        x1, y1, x2, y2 = int(max(0, x1)), int(max(0, y1)), int(min(width, x2)), int(min(height, y2))

        # Skip very small crops to avoid unclear images
        if (x2 - x1) < 20 or (y2 - y1) < 20:
            print(f"Skipped very small crop {i}")
            continue

        # Crop the detected region from the original RGB image
        cropped_img = img_rgb[y1:y2, x1:x2]

        # Display cropped image
        plt.figure(figsize=(4,3))
        plt.imshow(cropped_img)
        plt.axis('off')

        # Get class id and confidence score
        cls_id = int(box.cls[0])
        conf = box.conf[0]
        class_name = model.names[cls_id]

        plt.title(f"{class_name} ({conf:.2f})")
        plt.show()

Output hidden; open in https://colab.research.google.com to view.